In [80]:
# %pip install git+https://github.com/tensorflow/docs

import requests
import os.path
import asyncio
import json
import numpy as np
import pathlib
import pandas as pd
import seaborn as sns
import tensorflow as tf
import time
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
from riotwatcher import TftWatcher, ApiError

from utils.configuration import settings
from utils.utils import *
from tft.api import *


In [81]:
API_KEY = settings.api_key
ASSETS_DIR = settings.assets_dir
SERVER = 'na1'  # euw1 na1 kr oc1
LEAGUE='challengers'
MAX_COUNT = settings.max_count
LATEST_RELEASE = '12.12.450.4196' # '12.12.450.4196'

In [82]:
summoners_df = pd.read_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_summoners.pickle'))

In [83]:
summoners_df

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,MFA9xDr1qpMHq3p1P3Kd7yQRjmVZZeKgiEGMNOZaoHvSS_A,vNJAtsWbXGLyDPv5HWK6T2VsVvgaY3O7LOUNvL23vJt1-Q,hkhRHja9YNEGxs-c2YZxHJmvTcZaPQSaAMHWnbYt0qwlEC...,Pawnup,659,1656473052000,140,MFA9xDr1qpMHq3p1P3Kd7yQRjmVZZeKgiEGMNOZaoHvSS_A,Pawnup,730,I,167,114,False,False,True,False
1,cGaK7mqG4DmTXEga7Qy5ucEQ0Oxk4d9GhHnbQIEdMy2nUhU,q1c-kC9Zu8PLkjercNoH0nKHHG2K8Eclk4uel3tQp0GkRXA,P9mzq5hV79JOX9E7jKeIMG1U6kg8EHszz9z74FCsW7k8X3...,iG DQA,4275,1656466194000,240,cGaK7mqG4DmTXEga7Qy5ucEQ0Oxk4d9GhHnbQIEdMy2nUhU,iG DQA,883,I,139,100,False,False,False,False
2,kO9EW950IW3GG2BJMM2nZXV0X9XljZh4tmMyBCKI10THSZ4,L_AYRHh0Kbg2yETEfTO0wuPfAPBPhI8yH8tE-qoAlbI-vSg,j9twvE0iouy-mU175H0QaAVjr1hlhR_3IMXz9GZQmxOyXd...,OG Fuzi,3634,1656386952000,508,kO9EW950IW3GG2BJMM2nZXV0X9XljZh4tmMyBCKI10THSZ4,OG Fuzi,509,I,70,31,False,False,False,True
3,gGlqq3eNkHSc64jig-EnJiLU_mq4nwXS_upiVGnwW5NaeloW,y93jpEucG-0s3fnks3mSTcDOzUL3hRXErfv2QEQfl6av1k...,BI91yGTJgny887gEM0Z1-mgsYG_0qMRWvtZYrFgN8VgYgZ...,uwudamndaniel,5371,1656466004000,119,gGlqq3eNkHSc64jig-EnJiLU_mq4nwXS_upiVGnwW5NaeloW,uwudamndaniel,489,I,80,46,False,False,True,False
4,Eq_CljTi_Y8j1dFJ-rbumw0fQWF9Z45BTiZk4y6XTonW5MY,rqdNcntbVbnWo7aDM7Tml1OC4-HEmz-OhDNckky81CZ7BA,aYMQ0HjoMOURgWcc1NKogqLfUvAUQGKQb58HGfBvPZhuC1...,SpicyAppies,3591,1656472743000,37,Eq_CljTi_Y8j1dFJ-rbumw0fQWF9Z45BTiZk4y6XTonW5MY,SpicyAppies,1076,I,149,62,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,eMsAs5_WaPruim7bv3VhtHfBMltYhTQ9999-LcUVeU-cKYk,6duG4CESG_NemTQNUz_rdlNodWrgFO6Useh6XI6YOkbjTA,1mDiWGMTMZh3IFTiVjFXmhl53hq0hiHFitXwP_MBee5i1c...,Kyivix,547,1656468804000,146,eMsAs5_WaPruim7bv3VhtHfBMltYhTQ9999-LcUVeU-cKYk,Kyivix,693,I,112,72,False,False,True,True
81,mn3OHPDTDc5CsloPPGlHOQOq0Vqagk66ReC61-8SuXUxqH0,K4FBeziXvtE_px9iFYbp72rXuEhJO_W21lbvRF5VgM4Spw,502mQ2ZxwRwZDIiwzDF_nEvUBsq8qhU9kG_WayPz4qz9DE...,BC Socks,3552,1656466004000,74,mn3OHPDTDc5CsloPPGlHOQOq0Vqagk66ReC61-8SuXUxqH0,BC Socks,1243,I,149,72,True,False,False,False
82,3VzJ3Q9GpLuiqBdQXwztS_N_rHjUyruDavcA26BXHY8sua...,GnVen_Bb2diCno9OIsEsqBg3fj6yRfzlSgd6LmGJh1AEHG...,57iE8KPSssuk3oF_tlQfnWGn2RPChD1fAXIhw5ayRxp45K...,robivankenob,4986,1656472057000,67,3VzJ3Q9GpLuiqBdQXwztS_N_rHjUyruDavcA26BXHY8sua...,robivankenob,477,I,193,160,False,False,True,False
83,hlIh4LPuDu5w6Bi6mXiHUiQgjT7k4lSLWYAiGV174qRwfmQ,WRSxzgkWddIPZN0SQQS173iydk0H19sfBvHocZa1Dg94AH8,VylCtnjOgs2k9ctgMYVZB7G3buDEeryO2T9S1ejRTMWpqc...,TTV thekrayc,4780,1656450678000,209,hlIh4LPuDu5w6Bi6mXiHUiQgjT7k4lSLWYAiGV174qRwfmQ,TTV thekrayc,597,I,150,123,False,False,True,False


In [84]:

# Get all unique matches_id from assets dir
matches_asset = load_matches(summoners_df, server=SERVER)
matches_id = [match['metadata']['match_id'] for match in matches_asset]
seen = set()
uniq_matches_id = [
    x for x in matches_id if x not in seen and not seen.add(x)]

In [85]:
len(uniq_matches_id)

2969

In [86]:
len(matches_asset)

3823

In [87]:
seen = set()
seen_add = seen.add
uniq_matches = [x for x in matches_asset if x['metadata']['match_id'] not in seen and not seen_add(x['metadata']['match_id'])]

In [88]:
len(uniq_matches)

2969

In [89]:
matches_asset[150]['info']['game_version']

'Version 12.11.446.9344 (Jun 09 2022/13:23:39) [PUBLIC] <Releases/12.11>'

In [90]:
latest_matches = [match for match in uniq_matches if LATEST_RELEASE in match['info']['game_version']]

In [91]:
len(latest_matches)

1077

In [92]:
# latest_matches[0]['info']

In [93]:
matches_array = []

for match_row in latest_matches:
    match_id = match_row['metadata']['match_id']

    for participant in match_row['info']['participants']:
        match = {}
        match['match_id'] = match_id
        # match['level'] = participant['level']
        match['placement'] = participant['placement']
        # match['players_eliminated'] = participant['players_eliminated']
        # match['total_damage_to_players'] = participant['total_damage_to_players']

        for augment_index, augment in enumerate(participant['augments']):
            if augment == 'TFT7_Augment_GuildLootHR':
                augment = 'TFT7_Augment_BandOfThieves1'
            match[f'augment{augment_index}'] = augment

        for trait_index, trait in enumerate(participant['traits']):
            match[f'{trait["name"]}'] = trait["tier_current"]

        for unit_index, unit in enumerate(participant['units']):
            match[f'{unit["character_id"]}'] = unit["tier"]
            for item_index, item in enumerate(unit['itemNames']):
                match[f'{unit["character_id"]}_item{item_index}'] = item

        matches_array.append(match)

In [94]:
matches_array[0]
# len(matches_array)

{'match_id': 'NA1_4357487638',
 'placement': 5,
 'augment0': 'TFT6_Augment_PortableForge',
 'augment1': 'TFT7_Augment_DragonHorde',
 'augment2': 'TFT7_Augment_WhispersEmblem2',
 'Set7_Bruiser': 1,
 'Set7_Cavalier': 0,
 'Set7_Dragon': 2,
 'Set7_Guardian': 0,
 'Set7_Guild': 1,
 'Set7_Mage': 0,
 'Set7_Ragewing': 1,
 'Set7_Swiftshot': 1,
 'Set7_Warrior': 0,
 'Set7_Whispers': 3,
 'TFT7_Thresh': 1,
 'TFT7_Thresh_item0': 'TFT_Item_Zephyr',
 'TFT7_Shen': 2,
 'TFT7_Twitch': 1,
 'TFT7_Twitch_item0': 'TFT_Item_PowerGauntlet',
 'TFT7_Sylas': 2,
 'TFT7_Sylas_item0': 'TFT_Item_ThiefsGloves',
 'TFT7_Sylas_item1': 'TFT_Item_Zephyr',
 'TFT7_Sylas_item2': 'TFT_Item_GuardianAngel',
 'TFT7_Hecarim': 2,
 'TFT7_Hecarim_item0': 'TFT_Item_GuardianAngel',
 'TFT7_Hecarim_item1': 'TFT_Item_TearOfTheGoddess',
 'TFT7_Hecarim_item2': 'TFT_Item_JeweledGauntlet',
 'TFT7_Xayah': 2,
 'TFT7_Xayah_item0': 'TFT_Item_GuinsoosRageblade',
 'TFT7_Xayah_item1': 'TFT7_Item_WhispersEmblemItem',
 'TFT7_Xayah_item2': 'TFT_Item_Gua

In [95]:
matches_league_df = pd.json_normalize(matches_array)

In [96]:
matches_league_df

,match_id,placement,augment0,augment1,augment2,Set7_Bruiser,Set7_Cavalier,Set7_Dragon,Set7_Guardian,Set7_Guild,...,TFT7_TahmKench_item2,TFT7_Senna_item0,TFT7_Vladimir_item2,TFT7_Heimerdinger_item1,TFT7_Heimerdinger_item2,TFT7_Aatrox_item0,TFT7_Aatrox_item1,TFT7_Aatrox_item2,TFT7_Senna_item1,TFT7_Senna_item2
0,NA1_4357487638,5,TFT6_Augment_PortableForge,TFT7_Augment_DragonHorde,TFT7_Augment_WhispersEmblem2,1.0,0.0,2.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NA1_4357487638,3,TFT6_Augment_ThreesCompany,TFT7_Augment_FirstAidKit2,TFT6_Augment_TheGoldenEgg,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NA1_4357487638,6,TFT6_Augment_TrueTwos,TFT6_Augment_PortableForge,TFT6_Augment_VerdantVeil,NaN,NaN,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NA1_4357487638,7,TFT6_Augment_MetabolicAccelerator,TFT7_Augment_SwiftshotPressTheAttack,TFT7_Augment_BruiserEmblem2,3.0,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NA1_4357487638,8,TFT7_Augment_BruiserPersonalTraining,TFT7_Augment_WarriorTiamat,TFT6_Augment_VerdantVeil,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8611,NA1_4356117862,4,TFT6_Augment_TargetDummies,TFT7_Augment_ShapeshifterTrait,TFT6_Augment_JeweledLotus,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8612,NA1_4356117862,2,TFT6_Augment_PortableForge,TFT7_Augment_DragonmancerTrait,TFT6_Augment_Ascension,0.0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8613,NA1_4356117862,1,TFT6_Augment_SecondWind2,TFT7_Augment_BandOfThieves1,TFT6_Augment_Ascension,NaN,NaN,1.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8614,NA1_4356117862,8,TFT6_Augment_RichGetRicher,TFT6_Augment_ItemGrabBag1,TFT6_Augment_TargetDummies,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [97]:
matches_league_df[matches_league_df.isnull().T.any()]

,match_id,placement,augment0,augment1,augment2,Set7_Bruiser,Set7_Cavalier,Set7_Dragon,Set7_Guardian,Set7_Guild,...,TFT7_TahmKench_item2,TFT7_Senna_item0,TFT7_Vladimir_item2,TFT7_Heimerdinger_item1,TFT7_Heimerdinger_item2,TFT7_Aatrox_item0,TFT7_Aatrox_item1,TFT7_Aatrox_item2,TFT7_Senna_item1,TFT7_Senna_item2
0,NA1_4357487638,5,TFT6_Augment_PortableForge,TFT7_Augment_DragonHorde,TFT7_Augment_WhispersEmblem2,1.0,0.0,2.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NA1_4357487638,3,TFT6_Augment_ThreesCompany,TFT7_Augment_FirstAidKit2,TFT6_Augment_TheGoldenEgg,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NA1_4357487638,6,TFT6_Augment_TrueTwos,TFT6_Augment_PortableForge,TFT6_Augment_VerdantVeil,NaN,NaN,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NA1_4357487638,7,TFT6_Augment_MetabolicAccelerator,TFT7_Augment_SwiftshotPressTheAttack,TFT7_Augment_BruiserEmblem2,3.0,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NA1_4357487638,8,TFT7_Augment_BruiserPersonalTraining,TFT7_Augment_WarriorTiamat,TFT6_Augment_VerdantVeil,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8611,NA1_4356117862,4,TFT6_Augment_TargetDummies,TFT7_Augment_ShapeshifterTrait,TFT6_Augment_JeweledLotus,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8612,NA1_4356117862,2,TFT6_Augment_PortableForge,TFT7_Augment_DragonmancerTrait,TFT6_Augment_Ascension,0.0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8613,NA1_4356117862,1,TFT6_Augment_SecondWind2,TFT7_Augment_BandOfThieves1,TFT6_Augment_Ascension,NaN,NaN,1.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8614,NA1_4356117862,8,TFT6_Augment_RichGetRicher,TFT6_Augment_ItemGrabBag1,TFT6_Augment_TargetDummies,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
matches_league_df.to_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_matches.pickle'))